In [1]:
import pymongo
from pymongo import MongoClient
from bson import json_util
import json
import pprint
import datetime
from pytz import timezone
import pytz
import bson
from bson.objectid import ObjectId

In [2]:
client=MongoClient()
db=client['tweetsdb']
tweets=db['tweets']
pymongo.version

'3.7.2'

In [4]:
# with open('realdonaldtrump.json') as f:
#     file_data = json.load(f)
#     tweets.insert_many(file_data)


In [5]:
# client.close()

In [11]:
#this function updates all fields with the separated parts
def insertupdate(id_t,tweet_date):
    try:
        tweets.update_one({"_id":ObjectId(f"{id_t}")},{"$set":{"created_at_parts":tweet_date}})
    except Exception as e:
        print(e)
    else:
        print("no errors")
#     tweets.insert_one({})



def convertoint(id_t,tweet_date):
    try:
        tweets.update_one({"_id":ObjectId(f"{id_t}")},{"$set":{"created_at_parts.Day":tweet_date}})
    except Exception as e:
        print(e)

def insertime(id_t,tweet_time):
    try:
        tweets.update_one({"_id":ObjectId(f"{id_t}")},{"$set":{"created_at_parts.time":tweet_time}})
    except Exception as e:
        print(e)
    

In [13]:
# #this block is to separate date string into separate parts
created_at=tweets.find({},{"created_at":1,"_id":1})

for twts in created_at:
    date=datetime.datetime.strptime(twts["created_at"],'%a %b %d %H:%M:%S +0000 %Y').replace(tzinfo=pytz.UTC)
    dict_one={"Month":date.strftime("%b"),"Weekday":date.strftime("%a"),"Day":int(date.strftime("%d")),"Year":date.year}
    intday=int(date.strftime("%d"))
    tweet_id=twts["_id"]
    time=date.strftime("%H:%M:%S")
#     insertime(tweet_id,time)
#     convertoint(tweet_id,intday)
#     insertupdate(tweet_id,dict_one)


In [14]:
data=tweets.find({},{"created_at_parts.time":1})
for twts in data:
    pprint.pprint(twts)


{'_id': ObjectId('5c472b55ab09a13be42c2be7'),
 'created_at_parts': {'time': '11:28:45'}}
{'_id': ObjectId('5c472b55ab09a13be42c2be8'),
 'created_at_parts': {'time': '14:09:10'}}
{'_id': ObjectId('5c472b55ab09a13be42c2be9'),
 'created_at_parts': {'time': '13:12:48'}}
{'_id': ObjectId('5c472b55ab09a13be42c2bea'),
 'created_at_parts': {'time': '03:05:45'}}
{'_id': ObjectId('5c472b55ab09a13be42c2beb'),
 'created_at_parts': {'time': '13:03:57'}}
{'_id': ObjectId('5c472b55ab09a13be42c2bec'),
 'created_at_parts': {'time': '11:46:09'}}
{'_id': ObjectId('5c472b55ab09a13be42c2bed'),
 'created_at_parts': {'time': '00:25:53'}}
{'_id': ObjectId('5c472b55ab09a13be42c2bee'),
 'created_at_parts': {'time': '02:35:28'}}
{'_id': ObjectId('5c472b55ab09a13be42c2bef'),
 'created_at_parts': {'time': '18:06:30'}}
{'_id': ObjectId('5c472b55ab09a13be42c2bf0'),
 'created_at_parts': {'time': '12:29:48'}}
{'_id': ObjectId('5c472b55ab09a13be42c2bf1'),
 'created_at_parts': {'time': '14:06:06'}}
{'_id': ObjectId('5c4

 'created_at_parts': {'time': '12:06:55'}}
{'_id': ObjectId('5c472b55ab09a13be42c2d45'),
 'created_at_parts': {'time': '12:55:35'}}
{'_id': ObjectId('5c472b55ab09a13be42c2d46'),
 'created_at_parts': {'time': '19:02:05'}}
{'_id': ObjectId('5c472b55ab09a13be42c2d47'),
 'created_at_parts': {'time': '03:45:38'}}
{'_id': ObjectId('5c472b55ab09a13be42c2d48'),
 'created_at_parts': {'time': '15:24:37'}}
{'_id': ObjectId('5c472b55ab09a13be42c2d49'),
 'created_at_parts': {'time': '04:28:09'}}
{'_id': ObjectId('5c472b55ab09a13be42c2d4a'),
 'created_at_parts': {'time': '18:21:48'}}
{'_id': ObjectId('5c472b55ab09a13be42c2d4b'),
 'created_at_parts': {'time': '18:56:56'}}
{'_id': ObjectId('5c472b55ab09a13be42c2d4c'),
 'created_at_parts': {'time': '11:11:51'}}
{'_id': ObjectId('5c472b55ab09a13be42c2d4d'),
 'created_at_parts': {'time': '12:12:55'}}
{'_id': ObjectId('5c472b55ab09a13be42c2d4e'),
 'created_at_parts': {'time': '11:57:00'}}
{'_id': ObjectId('5c472b55ab09a13be42c2d4f'),
 'created_at_parts': 

 'created_at_parts': {'time': '01:51:10'}}
{'_id': ObjectId('5c472b55ab09a13be42c2e0d'),
 'created_at_parts': {'time': '20:56:34'}}
{'_id': ObjectId('5c472b55ab09a13be42c2e0e'),
 'created_at_parts': {'time': '18:57:07'}}
{'_id': ObjectId('5c472b55ab09a13be42c2e0f'),
 'created_at_parts': {'time': '19:03:57'}}
{'_id': ObjectId('5c472b55ab09a13be42c2e10'),
 'created_at_parts': {'time': '16:43:34'}}
{'_id': ObjectId('5c472b55ab09a13be42c2e11'),
 'created_at_parts': {'time': '00:37:49'}}
{'_id': ObjectId('5c472b55ab09a13be42c2e12'),
 'created_at_parts': {'time': '02:18:51'}}
{'_id': ObjectId('5c472b55ab09a13be42c2e13'),
 'created_at_parts': {'time': '20:41:45'}}
{'_id': ObjectId('5c472b55ab09a13be42c2e14'),
 'created_at_parts': {'time': '23:17:24'}}
{'_id': ObjectId('5c472b55ab09a13be42c2e15'),
 'created_at_parts': {'time': '08:08:52'}}
{'_id': ObjectId('5c472b55ab09a13be42c2e16'),
 'created_at_parts': {'time': '23:49:25'}}
{'_id': ObjectId('5c472b55ab09a13be42c2e17'),
 'created_at_parts': 

 'created_at_parts': {'time': '11:41:25'}}
{'_id': ObjectId('5c472b55ab09a13be42c2f39'),
 'created_at_parts': {'time': '20:48:12'}}
{'_id': ObjectId('5c472b55ab09a13be42c2f3a'),
 'created_at_parts': {'time': '19:51:41'}}
{'_id': ObjectId('5c472b55ab09a13be42c2f3b'),
 'created_at_parts': {'time': '00:31:40'}}
{'_id': ObjectId('5c472b55ab09a13be42c2f3c'),
 'created_at_parts': {'time': '17:08:29'}}
{'_id': ObjectId('5c472b55ab09a13be42c2f3d'),
 'created_at_parts': {'time': '15:39:43'}}
{'_id': ObjectId('5c472b55ab09a13be42c2f3e'),
 'created_at_parts': {'time': '11:37:46'}}
{'_id': ObjectId('5c472b55ab09a13be42c2f3f'),
 'created_at_parts': {'time': '15:21:58'}}
{'_id': ObjectId('5c472b55ab09a13be42c2f40'),
 'created_at_parts': {'time': '11:32:32'}}
{'_id': ObjectId('5c472b55ab09a13be42c2f41'),
 'created_at_parts': {'time': '11:05:16'}}
{'_id': ObjectId('5c472b55ab09a13be42c2f42'),
 'created_at_parts': {'time': '22:31:27'}}
{'_id': ObjectId('5c472b55ab09a13be42c2f43'),
 'created_at_parts': 

 'created_at_parts': {'time': '02:15:37'}}
{'_id': ObjectId('5c472b55ab09a13be42c3066'),
 'created_at_parts': {'time': '18:03:15'}}
{'_id': ObjectId('5c472b55ab09a13be42c3067'),
 'created_at_parts': {'time': '02:04:03'}}
{'_id': ObjectId('5c472b55ab09a13be42c3068'),
 'created_at_parts': {'time': '17:11:17'}}
{'_id': ObjectId('5c472b55ab09a13be42c3069'),
 'created_at_parts': {'time': '22:59:37'}}
{'_id': ObjectId('5c472b55ab09a13be42c306a'),
 'created_at_parts': {'time': '13:05:18'}}
{'_id': ObjectId('5c472b55ab09a13be42c306b'),
 'created_at_parts': {'time': '01:47:25'}}
{'_id': ObjectId('5c472b55ab09a13be42c306c'),
 'created_at_parts': {'time': '12:31:02'}}
{'_id': ObjectId('5c472b55ab09a13be42c306d'),
 'created_at_parts': {'time': '12:22:11'}}
{'_id': ObjectId('5c472b55ab09a13be42c306e'),
 'created_at_parts': {'time': '02:50:19'}}
{'_id': ObjectId('5c472b55ab09a13be42c306f'),
 'created_at_parts': {'time': '14:52:52'}}
{'_id': ObjectId('5c472b55ab09a13be42c3070'),
 'created_at_parts': 

{'_id': ObjectId('5c472b55ab09a13be42c3191'),
 'created_at_parts': {'time': '18:09:17'}}
{'_id': ObjectId('5c472b55ab09a13be42c3192'),
 'created_at_parts': {'time': '00:48:11'}}
{'_id': ObjectId('5c472b55ab09a13be42c3193'),
 'created_at_parts': {'time': '10:50:28'}}
{'_id': ObjectId('5c472b55ab09a13be42c3194'),
 'created_at_parts': {'time': '20:53:51'}}
{'_id': ObjectId('5c472b55ab09a13be42c3195'),
 'created_at_parts': {'time': '01:52:50'}}
{'_id': ObjectId('5c472b55ab09a13be42c3196'),
 'created_at_parts': {'time': '00:49:27'}}
{'_id': ObjectId('5c472b55ab09a13be42c3197'),
 'created_at_parts': {'time': '21:09:16'}}
{'_id': ObjectId('5c472b55ab09a13be42c3198'),
 'created_at_parts': {'time': '13:14:13'}}
{'_id': ObjectId('5c472b55ab09a13be42c3199'),
 'created_at_parts': {'time': '13:05:34'}}
{'_id': ObjectId('5c472b55ab09a13be42c319a'),
 'created_at_parts': {'time': '00:20:53'}}
{'_id': ObjectId('5c472b55ab09a13be42c319b'),
 'created_at_parts': {'time': '13:56:12'}}
{'_id': ObjectId('5c4

{'_id': ObjectId('5c472b55ab09a13be42c32bd'),
 'created_at_parts': {'time': '11:13:26'}}
{'_id': ObjectId('5c472b55ab09a13be42c32be'),
 'created_at_parts': {'time': '20:12:15'}}
{'_id': ObjectId('5c472b55ab09a13be42c32bf'),
 'created_at_parts': {'time': '14:15:07'}}
{'_id': ObjectId('5c472b55ab09a13be42c32c0'),
 'created_at_parts': {'time': '11:39:53'}}
{'_id': ObjectId('5c472b55ab09a13be42c32c1'),
 'created_at_parts': {'time': '13:36:25'}}
{'_id': ObjectId('5c472b55ab09a13be42c32c2'),
 'created_at_parts': {'time': '20:06:53'}}
{'_id': ObjectId('5c472b55ab09a13be42c32c3'),
 'created_at_parts': {'time': '11:23:00'}}
{'_id': ObjectId('5c472b55ab09a13be42c32c4'),
 'created_at_parts': {'time': '12:42:52'}}
{'_id': ObjectId('5c472b55ab09a13be42c32c5'),
 'created_at_parts': {'time': '18:10:09'}}
{'_id': ObjectId('5c472b55ab09a13be42c32c6'),
 'created_at_parts': {'time': '01:10:39'}}
{'_id': ObjectId('5c472b55ab09a13be42c32c7'),
 'created_at_parts': {'time': '18:03:29'}}
{'_id': ObjectId('5c4

{'_id': ObjectId('5c472b55ab09a13be42c33e9'),
 'created_at_parts': {'time': '12:31:18'}}
{'_id': ObjectId('5c472b55ab09a13be42c33ea'),
 'created_at_parts': {'time': '13:30:44'}}
{'_id': ObjectId('5c472b55ab09a13be42c33eb'),
 'created_at_parts': {'time': '13:08:24'}}
{'_id': ObjectId('5c472b55ab09a13be42c33ec'),
 'created_at_parts': {'time': '10:33:44'}}
{'_id': ObjectId('5c472b55ab09a13be42c33ed'),
 'created_at_parts': {'time': '01:29:09'}}
{'_id': ObjectId('5c472b55ab09a13be42c33ee'),
 'created_at_parts': {'time': '23:18:00'}}
{'_id': ObjectId('5c472b55ab09a13be42c33ef'),
 'created_at_parts': {'time': '13:55:59'}}
{'_id': ObjectId('5c472b55ab09a13be42c33f0'),
 'created_at_parts': {'time': '17:04:04'}}
{'_id': ObjectId('5c472b55ab09a13be42c33f1'),
 'created_at_parts': {'time': '17:22:35'}}
{'_id': ObjectId('5c472b55ab09a13be42c33f2'),
 'created_at_parts': {'time': '03:33:30'}}
{'_id': ObjectId('5c472b55ab09a13be42c33f3'),
 'created_at_parts': {'time': '20:23:58'}}
{'_id': ObjectId('5c4

{'_id': ObjectId('5c472b55ab09a13be42c3515'),
 'created_at_parts': {'time': '04:08:45'}}
{'_id': ObjectId('5c472b55ab09a13be42c3516'),
 'created_at_parts': {'time': '01:00:41'}}
{'_id': ObjectId('5c472b55ab09a13be42c3517'),
 'created_at_parts': {'time': '11:00:14'}}
{'_id': ObjectId('5c472b55ab09a13be42c3518'),
 'created_at_parts': {'time': '15:15:21'}}
{'_id': ObjectId('5c472b55ab09a13be42c3519'),
 'created_at_parts': {'time': '18:31:07'}}
{'_id': ObjectId('5c472b55ab09a13be42c351a'),
 'created_at_parts': {'time': '03:14:02'}}
{'_id': ObjectId('5c472b55ab09a13be42c351b'),
 'created_at_parts': {'time': '02:15:03'}}
{'_id': ObjectId('5c472b55ab09a13be42c351c'),
 'created_at_parts': {'time': '02:38:54'}}
{'_id': ObjectId('5c472b55ab09a13be42c351d'),
 'created_at_parts': {'time': '20:34:20'}}
{'_id': ObjectId('5c472b55ab09a13be42c351e'),
 'created_at_parts': {'time': '18:22:12'}}
{'_id': ObjectId('5c472b55ab09a13be42c351f'),
 'created_at_parts': {'time': '13:39:54'}}
{'_id': ObjectId('5c4

{'_id': ObjectId('5c472b55ab09a13be42c3641'),
 'created_at_parts': {'time': '19:05:39'}}
{'_id': ObjectId('5c472b55ab09a13be42c3642'),
 'created_at_parts': {'time': '04:57:57'}}
{'_id': ObjectId('5c472b55ab09a13be42c3643'),
 'created_at_parts': {'time': '15:38:49'}}
{'_id': ObjectId('5c472b55ab09a13be42c3644'),
 'created_at_parts': {'time': '12:11:01'}}
{'_id': ObjectId('5c472b55ab09a13be42c3645'),
 'created_at_parts': {'time': '12:47:07'}}
{'_id': ObjectId('5c472b55ab09a13be42c3646'),
 'created_at_parts': {'time': '22:14:23'}}
{'_id': ObjectId('5c472b55ab09a13be42c3647'),
 'created_at_parts': {'time': '22:53:07'}}
{'_id': ObjectId('5c472b55ab09a13be42c3648'),
 'created_at_parts': {'time': '12:39:18'}}
{'_id': ObjectId('5c472b55ab09a13be42c3649'),
 'created_at_parts': {'time': '09:53:41'}}
{'_id': ObjectId('5c472b55ab09a13be42c364a'),
 'created_at_parts': {'time': '02:40:23'}}
{'_id': ObjectId('5c472b55ab09a13be42c364b'),
 'created_at_parts': {'time': '00:42:49'}}
{'_id': ObjectId('5c4

In [169]:

results=tweets.find_one({"created_at_parts.Month":"Jan"})
results

{'_id': ObjectId('5c472b55ab09a13be42c2c01'),
 'created_at': 'Thu Jan 18 01:05:46 +0000 2018',
 'id': 953795518592843777,
 'id_str': '953795518592843777',
 'full_text': 'Despite some very corrupt and dishonest media coverage, there are many great reporters I respect and lots of GOOD NEWS for the American people to be proud of!',
 'truncated': False,
 'display_text_range': [0, 158],
 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []},
 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 25073877,
  'id_str': '25073877',
  'name': 'Donald J. Trump',
  'screen_name': 'realDonaldTrump',
  'location': 'Washington, DC',
  'description': '45th President of the United States of America🇺🇸',
  'url': 'https://t.co/OMxB0x7xC5',
  'entities': {'url':

In [164]:
data()